<div align="right">Naoaki ONO, Shigehiko KANAYA <br/>
NAIST DSC</div>

[Open in Google Colaboratory](https://colab.research.google.com/github/naono-git/colaboratory/blob/master/note_02_hello_TF.ipynb)

# はじめてのTensorFlow



TensorFlowはGoogleが主導になってオープンソースで開発が進められている深層学習のためのライブラリです。

自分でゼロからインストールしようとするとそれなりに手間がかかりますが、幸いColaboratoryでは最新版がインストールされた環境が整えられています。

現在のバージョンは下記の変数で確認できます。

In [0]:
import tensorflow as tf
print(tf.__version__)

1.13.0-rc1


## Define and Run

Pythonは本来「インタプリタ型」言語ですのでプログラムは逐次的に実行されるのですが、そのままだと最適化ができないため計算速度が遅くなるという問題があります。

TensorFlowでは計算の高速化のため「はじめにニューラルネットワークの計算手順を全部定義しておき、最適化した後に計算する」という手法をとっています。先に計算を定義するので"Define-and-run"モデルと呼ばれています。

プログラミングに若干手間がかかる代わり、自動的にGPUに最適化して高速に計算してくれます。

（"Define-and-run"に対して、Chainer, PyTorchなどのライブラリのように、入力データに対して動的にネットワークを構築していくことができる"Define-by-run"と呼ばれるモデルも開発されています。TensorFlowもバージョン2.0ではこれに対応するとの噂です）




試しに「100次元のベクトルに100x100の行列を掛け、平均する、のを100万回計算する」時間を測ってみましょう。

先に乱数でデータを作っておきます。

In [0]:
import numpy as np

nd = 100
nn = 1000000
xxx = np.reshape(np.random.normal(10, 1, nd*nn), (nn,nd))
www = np.reshape(np.random.normal(0, 1, nd*nd), (nd,nd))

### foolish-loop

あまり効率の良い方法ではないですが(注)、単純に100万回のループを繰り返してみましょう。

さすがにちょっと待たされます。

`time()`関数で現在時刻がわかるので計算前後の時刻を引き算すれば何秒かかったかをラフに測ることができます。

In [0]:
import time

yyy = np.zeros(nn, dtype=np.float)
t1 = time.time() 
for aa in range(nn):
  tmp = np.matmul(xxx[aa,:], www)
  yyy[aa] = np.mean(tmp)
t2 = time.time() 
print(t2-t1, "secs")

25.33461594581604 secs


### TensorFlow session

次にTensorFlowを使って計算してみましょう。

まず、TensorFlowでの計算手順を保存するためのsessionという環境を用意します。

これは、基本的には最初に一度だけ実行すれば問題ありません。

In [0]:
if not "sess" in globals():
  sess = tf.InteractiveSession()

次に、計算手順を定義します。

入力されるデータの形と、データに適用される一連の関数を与えます。

定義が終わったら一度「初期化」を行います。

この段階で`output1`を見てもまだ数値は出てきません。
「`tf.Tensor`」なるオブジェクトが定義されているだけです。

In [0]:

input1 = tf.placeholder(dtype=tf.float64, shape=(None, nd))
tfwww = tf.Variable(www,dtype=tf.float64)
layer1 = tf.tensordot(input1, tfwww, axes=[1,0])

output1 = tf.reduce_mean(layer1, axis=1)

sess.run(tf.global_variables_initializer())
output1


Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor 'Mean:0' shape=(?,) dtype=float64>

さてお待たせしました。ようやく計算できます。

`output1`の計算結果を得るために、`eval()`という関数を適用します。

今回、この`output1`の計算には引数となる入力データとして`input1`が使われているので、
ここに実際のデータを与える必要があります。

入力データの"変数名"と与える値をペアにした辞書オブジェクトの形式で与えるので"feed_dict"と呼ばれています。

今回は先ほど生成した乱数の配列`xxx`を使うので、引数は`feed_dict = {input1: xxx}`
となります。



In [0]:
t1 = time.time() 
result1 = output1.eval(feed_dict={input1: xxx})
t2 = time.time()
print(t2-t1, "secs")

0.8667111396789551 secs


 最適化された関数が実行されるので、先ほどより何十倍か早くなっています。

ちなみにオブジェクトに`eval()`関数を適用する代わりに`sess.run()`関数を使ってTensorFlowオブジェクトを評価しても同じように結果を取得できます。

複数のオブジェクトの計算結果をまとめて求めたい場合などはこちらの方が便利です。

In [0]:
t1 = time.time() 
res_layer1, res_out1 = sess.run((layer1, output1), feed_dict={input1: xxx})
t2 = time.time()
print(t2-t1, "secs")

0.7620186805725098 secs


補足：上の例では引数であるinput1が比較的すぐにoutput1の計算に使われていますが、ニューラルネットの計算では最後の値を得るために何段階も、何十段階もの関数が必要になることはざらにあります。

そのような場合でも、直接間接を問わず出力結果に影響がある入力データを全てこのfeed_dictのリストとして渡してあげる必要があります（逆に依存する入力データがないようなオブジェクトの場合にはfeed_dictは空でも構いません）

### バッチ入力

実際のニューラルネットの学習では、入力データの数が膨大になる場合がしばしばあります。

そのような場合には入力データを「バッチ」に分割して逐次的に計算することもあります。

単純に頭から順に`num_batch`個ずつデータを入力してもいいのですが、順番の偏りによるアーティファクトを避けるために繰り返しの度にランダムにシャッフルするのが一般的です。

バッチサイズは大きめにする方が計算効率は良くなりますが、GPUのメモリなどの限界もあるので100前後にすることが多いでしょうか。

In [0]:
nb = 10000
t1 = time.time() 
for aa in range(nn//nb):
  output1.eval({input1: xxx[(0+nb*aa):(nb+nb*aa),:]})
t2 = time.time() 
print(t2-t1, "secs")

0.3379063606262207 secs


### 注）Numpyの場合

正直なところを言えばこの程度の計算ならばTensorFlowでなくてもnumpyライブラリだけでもそこそこ高速に計算できます。

In [0]:
t1 = time.time() 
tmp = np.tensordot(xxx, www, axes=[1,0])
np.mean(tmp, axis=0)
t2 = time.time() 
print(t2-t1, "secs")

1.1847271919250488 secs
